In [ ]:
from sklearn.datasets import make_regression
import pandas as pd
import numpy as np
X,y= make_regression(n_samples=100,n_features=2,noise=10,random_state=42)
from sklearn.model_selection import train_test_split
X_train,x_test,Y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
# 1. varience threshold
from sklearn.feature_selection import VarianceThreshold
selector=VarianceThreshold(threshold=0.90)
selector.fit(X_train) # calculates the varience of each feature
selector.get_support() # gives which features have higher threshold
X_train_Transform = selector.transform(X_train) # select those features only with theier value

In [ ]:
# 3 Correlation
corr_matrix=pd.DataFrame(X_train).corr()
columns = corr_matrix.columns
columns_to_drop = []
for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        # Access the cell of the DataFrame
        if corr_matrix.loc[columns[i], columns[j]] > 0.95:
            columns_to_drop.append(columns[j])

print(len(columns_to_drop))

In [ ]:
# 4 Annova
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest

sel = SelectKBest(f_classif, k=100).fit(X_train, y_train)
X_train.columns[sel.get_support()]

In [ ]:
# 5 Chi-square Test
titanic = pd.read_csv('/kaggle/input/titanic/train.csv')[['Pclass','Sex','SibSp','Parch','Embarked','Survived']]
titanic.head()
ct = pd.crosstab(titanic['Survived'],titanic['Sex'],margins=True)
from scipy.stats import chi2_contingency
chi2_contingency(ct)
score = []
for feature in titanic.columns[:-1]:
    # create contingency table
    ct = pd.crosstab(titanic['Survived'], titanic[feature])
    # chi_test
    p_value = chi2_contingency(ct)[1]
    score.append(p_value)

    pd.Series(score, index=titanic.columns[:-1]).sort_values(ascending=True).plot(kind='bar')

# one more method
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt

# assuming titanic is your DataFrame and 'Survived' is the target column

# Encode categorical variables
le = LabelEncoder()
titanic_encoded = titanic.apply(le.fit_transform)

X = titanic_encoded.drop('Survived', axis=1)
y = titanic_encoded['Survived']

# Calculate chi-squared stats
chi_scores = chi2(X, y)

# chi_scores[1] are the p-values of each feature.
p_values = pd.Series(chi_scores[1], index = X.columns)
p_values.sort_values(inplace = True)

# Plotting the p-values
p_values.plot.bar()

plt.title('Chi-square test - P-values')
plt.xlabel('Feature')
plt.ylabel('P-value')

plt.show()